In [9]:
%load_ext autoreload
%autoreload 2

In [68]:
import gtsa
import xarray as xr

In [69]:
# clip by glacier
# get glacier centerline
# extract points
# sample along points
# fit regression to sampled time series
# fit regression to entire stack
# compute continuous volume change for each glacier

In [78]:
bucket = 'petrichor'
aws_server_url = 's3.us-west-2.amazonaws.com'
product = 'stack'
base_directory = './stacks'

sites = gtsa.io._get_test_sites(bucket)

for s in sites:
    folder = gtsa.io.Path(s, product).as_posix()
    
    zarr_url = gtsa.io.parse_urls_from_S3_bucket(bucket,
                                                 aws_server_url = aws_server_url,
                                                 folder = folder,
                                                 extension= 'stack.zarr')[0]
    print(s)
    print(len(ds.time))

In [79]:
ds = xr.open_dataset(zarr_url,chunks='auto',engine='zarr')

In [81]:
len(ds.time)

12